In [1]:
## import data manipulation packages for data cleaning and distance calculation
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric
from math import radians

In [2]:
## DATA CLEANING AND PREPARATION
## import dataset as variable 'city' and drop NaN
cities = pd.read_excel('data/worldcities.xlsx', engine = 'openpyxl')
ct = cities.dropna(axis = 'rows', how = 'any')

In [3]:
## add london starting point as 'London_st' slights on the right (to facilitate the assignment resolution)
London_st = ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]
London_st['city']='London_st' 
London_st['lng'] = London_st['lng'] + 0.2
ct = ct.append(London_st)

<ipython-input-3-bec5f70982d8>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['city']='London_st'
<ipython-input-3-bec5f70982d8>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  London_st['lng'] = London_st['lng'] + 0.2


In [4]:
## resetting index after append
ct = ct.reset_index()

In [5]:
## concatenate iso2 and city to get unique id
ct['ID'] = ct['city'].map(str) + ct['iso2'].map(str)

In [6]:
## drop not usable columns
ct = ct.drop(['city_ascii', 'country', 'iso2', 'admin_name', 'capital', 'id'], axis = 1)
ct = ct.drop('index', axis = 1)

In [7]:
## identifying location of 'London_st' to be used later as 'source'
source = ct.loc[(ct['city'] == 'London_st')]

In [8]:
## identifying location of 'London' to be used later as 'target'
target = ct.loc[(ct['city'] == 'London') & (ct['iso3'] == 'GBR')]

In [9]:
## GETTING WEIGHTS - part I
## population weights '+2', where population > 200000
pop = np.where(ct['population'] < 200000 , 0, 2)

In [10]:
## same state weights '+2', where 'iso3' is different
i = ct['iso3'].to_numpy()
st = (i[:, None ] != i) * 2

In [11]:
## GETTING DIRECTION - getting an array comparing longitudes (0 if a city is west the other, 1 if a city is east)
## to get all positive longitudes we need to rescale from -180/+180 to 0/360 scale, where London is approx 0
dr_x = np.where(ct['lng']>= 0 , ct['lng'] , (ct['lng'] + 180) + 180)
x = dr_x
dr = (x[:, None] < x) * 1

In [12]:
## computing big distances (>60 degrees) as a '0' (no go area) to get the final matrix less 'heavy' to be handled
rang = (x[: , None] < x + 60 ) * 1

In [13]:
## QUESTO NON SERVE GIUSTO?
## dir_test = pd.DataFrame(dr*rang.T, columns = ct['ID'], index = ct['ID'])
## dir_test

In [14]:
## creating 3 dataframes with direction, same state and population weights
direction = pd.DataFrame(dr*rang.T, columns = ct['ID'], index = ct['ID'])
same_state = pd.DataFrame(st, columns = ct['ID'], index = ct['ID'])
population = pd.DataFrame(pop , index = ct['ID'])

In [15]:
## DISTANCE COMPUTATION - 'Harvesine'
## the earth is spheric, so a specific calculation ('Harvesine distance') is required to get the distance from places
ct['lat'] = np.radians(ct['lat'])
ct['lng'] = np.radians(ct['lng'])

In [16]:
## retrieve the 'harvesine' metric from scipy
dist = DistanceMetric.get_metric('haversine')

In [17]:
## calculating the pairwise distance between cities multiplying *6373 to get kms
## get a smaller size object by getting distance only if direction is 'east' (value 1 in 'direction' dataframe)
D = np.where(direction > 0, dist.pairwise(ct [['lat','lng']].to_numpy())*6373 , 0)
## create the distance matrix with cities in the indexes
distance = pd.DataFrame(D.T, columns = ct['ID'], index = ct['ID'])

In [18]:
## view matrix of distance

In [19]:
## QUESTO NON SERVE GIUSTO?
## distance.loc['London_stGB'].sum()

In [20]:
## secondo me questo è già risolto con import pandas as pd no?
## from pandas import DataFrame

In [21]:
## GETTING WEIGHTS - part II
## utilising the matrix of distance called 'distance' (which contains already directions)
## populate 'dis' with weights: '+2' if closest, '4' if second closest, '8' if third closest
## the rest of distances as '0', meaning 'no go'
dis = distance.T.replace(0, 0)
dis = dis.replace(dis.apply(lambda x: x[x > 0].min(axis=0)), 2)
dis = dis.replace(dis.apply(lambda x: x[x > 2].min(axis=0)), 4)
dis = dis.replace(dis.apply(lambda x: x[x > 4].min(axis=0)), 8)
dis = dis.where((dis <= 8), 0) 
dis

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JakartaID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DelhiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MumbaiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ManilaPH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Río CuartoAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La EsmeraldaVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
## SUMMING THE TOTAL WEIGHTS
## sum of dataframes: 'dis', 'same_state' and 'population' to get final weights
graph =((dis + same_state + pop.T) * dis / dis)
graph = graph.where((graph > 1), 0) 
graph

ID,TokyoJP,JakartaID,DelhiIN,MumbaiIN,ManilaPH,ShanghaiCN,São PauloBR,SeoulKR,Mexico CityMX,GuangzhouCN,...,RodeoAR,GastreAR,TinogastaAR,TelsenAR,GreytownNI,SayḩūtYE,Río CuartoAR,La EsmeraldaVE,HödrögöMN,London_stGB
ID,,,,,,,,,,,,,,,,,,,,,
TokyoJP,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
JakartaID,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DelhiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MumbaiIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ManilaPH,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SayḩūtYE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Río CuartoAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La EsmeraldaVE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [23]:
## preparation of final dataframe as array for 'NetworkX' 
gr_array = np.array(graph)
gr_array

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
## SHORTEST PATH ALGORITHM aka Dijkstra's algorithm
## import NetworkX
import networkx as nx

In [25]:
## convert the numpy array to GRAPH data structure, with has nodes (cities) and edges (weights between nodes)
## zeros are not taken into account, so the direction is taken into account in the built array
GR = nx.from_numpy_array(gr_array)

In [26]:
## edges visualization (optional)
GR.edges(data=True)

EdgeDataView([(0, 585, {'weight': 4.0}), (0, 873, {'weight': 10.0}), (0, 150, {'weight': 6.0}), (0, 375, {'weight': 4.0}), (0, 1164, {'weight': 10.0}), (1, 232, {'weight': 4.0}), (1, 1710, {'weight': 8.0}), (1, 2206, {'weight': 4.0}), (1, 411, {'weight': 6.0}), (1, 783, {'weight': 4.0}), (1, 1691, {'weight': 10.0}), (2, 163, {'weight': 4.0}), (2, 2993, {'weight': 10.0}), (2, 177, {'weight': 10.0}), (2, 464, {'weight': 6.0}), (2, 546, {'weight': 4.0}), (3, 36, {'weight': 10.0}), (3, 542, {'weight': 6.0}), (3, 3099, {'weight': 2.0}), (3, 3117, {'weight': 2.0}), (3, 53, {'weight': 4.0}), (3, 1648, {'weight': 6.0}), (3, 4956, {'weight': 12.0}), (4, 187, {'weight': 4.0}), (4, 729, {'weight': 6.0}), (4, 803, {'weight': 6.0}), (4, 971, {'weight': 4.0}), (4, 1001, {'weight': 6.0}), (4, 1987, {'weight': 2.0}), (4, 317, {'weight': 4.0}), (4, 1076, {'weight': 6.0}), (4, 1276, {'weight': 10.0}), (5, 102, {'weight': 4.0}), (5, 590, {'weight': 10.0}), (5, 100, {'weight': 10.0}), (5, 310, {'weight': 

In [27]:
## nodes visualization (optional)
GR.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [28]:
## retrieve location of 'London_st' as source and 'London' as origin
print(source)
print(target)

           city      lat     lng iso3  population           ID
6622  London_st  51.5072  0.0725  GBR  10979000.0  London_stGB
      city      lat     lng iso3  population        ID
31  London  51.5072 -0.1275  GBR  10979000.0  LondonGB


In [29]:
## using networkx.single_source_dijkstra()
## the command computes shortest paths and lengths in a weighted graph G
## it returns a tuple containing the 'length' of the shortest path, and the 'path' itself
length, path = nx.single_source_dijkstra(GR, 6622, 31)
print(length, path)

278.0 [6622, 2064, 2669, 5987, 2254, 3013, 353, 1127, 6239, 4596, 5579, 6596, 6498, 3103, 5449, 6583, 5748, 6587, 1262, 3435, 2998, 2500, 685, 1557, 409, 319, 828, 1716, 2280, 6621, 3492, 3409, 3550, 4095, 3907, 3707, 3169, 4308, 4833, 1241, 2540, 3668, 3450, 1362, 1058, 439, 2645, 3710, 1298, 1980, 2874, 1889, 321, 3155, 2062, 977, 3959, 4018, 2739, 3343, 2829, 3483, 2980, 4430, 3028, 2799, 31]


In [30]:
## get the names of the 'path' retrieving from 'ct' original object
ct.loc[path, 'city']

6622           London_st
2064    Boulogne-sur-Mer
2669              Calais
5987              Veurne
2254              Bruges
              ...       
2980           Waterford
4430             Quimper
3028        Saint-Brieuc
2799           Cherbourg
31                London
Name: city, Length: 67, dtype: object

In [31]:
## quanti giorni per fare il giro del mondo?
days_to_london = length * 0.041667
days_to_london

11.583426000000001

In [45]:
##crea il dataframe con i dati del percorso partendo da cities perchè in ct le lat e lng sono state sovrascritte dai calcoli precedenti
lista=cities.iloc[path]
print(lista)

           city city_ascii      lat       lng         country iso2 iso3  \
6622     Belper     Belper  53.0290   -1.4750  United Kingdom   GB  GBR   
2064    Sekondi    Sekondi   4.9433   -1.7040           Ghana   GH  GHA   
2669  Charleroi  Charleroi  50.4167    4.4442         Belgium   BE  BEL   
5987      Bucha      Bucha  50.5464   30.2350         Ukraine   UA  UKR   
2254     Peoria     Peoria  40.7520  -89.6153   United States   US  USA   
...         ...        ...      ...       ...             ...  ...  ...   
2980     Zabrze     Zabrze  50.3000   18.7833          Poland   PL  POL   
4430     Sparks     Sparks  39.5729 -119.7157   United States   US  USA   
3028        Deo        Deo  24.6561   84.4356           India   IN  IND   
2799       Waco       Waco  31.5598  -97.1881   United States   US  USA   
31      Chengdu    Chengdu  30.6636  104.0667           China   CN  CHN   

             admin_name capital  population          id  
6622         Derbyshire     NaN     21822

In [34]:
#non so se necessario ma ordina le città 
#lista = lista.reset_index()

In [40]:
#import delle librerie per i grafici
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [41]:
##crea il primo grafico con le traiettorie tra le città sulla base della mappa mondiale
fig = go.Figure(data=go.Scattergeo(lat = lista['lat'],lon =lista['lng'],mode = 'lines',line = dict(width = 2, color = 'blue'),))

In [42]:
##aggiorna il grafico aggiungendo i marker per le città visitate con nome della città se selezionate con mouse
fig.add_trace(go.Scattergeo(
    locationmode = 'country names',
    lon = lista['lng'],
    lat = lista['lat'],
    hoverinfo = 'text',
    text = lista['city'],
    name = "Cities",
    mode = 'markers',
    marker = dict(
        size = 6,
        color = 'rgb(102,102,102)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))
fig.show()